In [1]:
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors as MLLibVectors
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml.feature import * 
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder


from nltk.stem.porter import *
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

#import pandas as pd
import string 
import re 

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1648904705274_0004,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("pandas==0.25.1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

You are using pip version 9.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [3]:
review = spark.read.parquet("s3://yelp-work-repo-backup-s26-urc/yelp_academic_dataset_review_parquet/*.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# remove punctuation
def remove_punct(text):
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text) 
    return nopunct
    
# binarize rating
def convert_rating(rating):
    rating = int(rating)
    if rating >=4: return 1
    else: return 0

# udf
punct_remover = udf(lambda x: remove_punct(x))
rating_convert = udf(lambda x: convert_rating(x))

# apply to review raw data
review_df = review.select(punct_remover('text'), rating_convert('stars'))

review_df = review_df.withColumnRenamed('<lambda>(text)', 'text')\
                     .withColumn('label', review_df["<lambda>(stars)"].cast(IntegerType()))\
                     .drop('<lambda>(stars)')\
                     .limit(5000)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# tokenize
tok = Tokenizer(inputCol="text", outputCol="words")
review_tokenized = tok.transform(review_df)

# remove stop words
stopword_rm = StopWordsRemover(inputCol='words', outputCol='words_nsw')
review_tokenized = stopword_rm.transform(review_tokenized)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
review_tokenized.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+--------------------+
|                text|label|               words|           words_nsw|
+--------------------+-----+--------------------+--------------------+
|Kelvin at Realty ...|    1|[kelvin, at, real...|[kelvin, realty, ...|
|My husband and I ...|    1|[my, husband, and...|[husband, big, fa...|
|I ve been to Tape...|    1|[i, ve, been, to,...|[ve, tapeo, coupl...|
|In town for NCAA ...|    1|[in, town, for, n...|[town, ncaa, lax,...|
|This is probably ...|    1|[this, is, probab...|[probably, cutest...|
|Buford Highway Fa...|    1|[buford, highway,...|[buford, highway,...|
|Delicious  I love...|    1|[delicious, , i, ...|[delicious, , lov...|
|Cannot say enough...|    1|[cannot, say, eno...|[say, enough, goo...|
|I recently discov...|    1|[i, recently, dis...|[recently, discov...|
|I see this restau...|    1|[i, see, this, re...|[see, restaurant,...|
|Finally got to tr...|    1|[finally, got, to...|[finally, got, tr...|
|Order

In [7]:
# count vectorizer
cv = CountVectorizer(inputCol='words_nsw', outputCol='tf')
cvModel = cv.fit(review_tokenized)
count_vectorized = cvModel.transform(review_tokenized)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.ml.feature import IDF
idf = IDF().setInputCol('tf').setOutputCol('tfidf')
tfidfModel = idf.fit(count_vectorized)
tfidf_df = tfidfModel.transform(count_vectorized)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
tfidf_df.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|                text|label|               words|           words_nsw|                  tf|               tfidf|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|Had to bring the ...|    0|[had, to, bring, ...|[bring, truck, ch...|(19144,[0,5,23,30...|(19144,[0,5,23,30...|
|Very friendly rec...|    0|[very, friendly, ...|[friendly, recept...|(19144,[0,26,28,1...|(19144,[0,26,28,1...|
+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows

In [9]:
neg_voc = spark.read.csv("s3://positiveandnegative/negative-words.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
pos_voc = spark.read.csv("s3://positiveandnegative/positive-words.txt")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
positive_list=(pos_voc.select('_c0').rdd.flatMap(lambda x: x).collect())
negative_list=(neg_voc.select('_c0').rdd.flatMap(lambda x: x).collect())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
########################################################################################################################

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df=review_tokenized.toPandas()
texts=[]
for review in df.words_nsw:
    texts.append(review)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
texts


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[['bring', 'truck', 'chevrolet', 'recall', '', '', 'came', 'pick', 'recall', 'resolved', 'problems', '', '', 'left', 'visit', '', '', '', 'yet', 'another', 'problem', 'didn', 'exist', '', '', 'horrible', 'knowledge', '', 'service', 'integrity', '', '', 'bring', 'business', '', '', 'll', 'sorry'], ['friendly', 'reception', 'staff', '', 'kind', 'educated', 'assistant', '', '', 'dr', 'stoever', 'extremely', 'kurt', '', 'return'], ['son', 'buying', 'house', 'locally', 'needed', 'organize', 'inspection', 'lives', 'state', '', 'chose', 'pro', 'tech', 'one', 'person', 'firm', '', 'wayne', '', 'proprietor', '', 'excellent', 'every', 'way', '', 'bit', 'nervous', 'inspection', 'house', 'flipped', 'contractor', 'original', 'structure', 'built', '', '', '', '', '', '', 'wanted', '', '', 'day', 'radon', 'test', 'continuous', 'read', 'outs', 'wayne', 'came', 'separately', 'set', 'followed', 'inspection', '', '', 'days', 'later', '', 'thorough', '', 'noting', 'many', 'things', 'work', 'owner', '', 'w

In [27]:
#texts

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
#####################################################################################################################

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
positive_negative=[]
for sentence in texts:
    for tokens in sentence:
        lp=[]
        ln=[]
        if tokens in positive_list:
            lp.append(tokens)
        if tokens in negative_list:
            ln.append(tokens)
    if len(lp)>len(ln):
        positive_negative.append(1)
    else:
        positive_negative.append(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
b = sqlContext.createDataFrame([(l,) for l in positive_negative], ['pn'])

b.shape()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'shape'
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py", line 1307, in __getattr__
    "'%s' object has no attribute '%s'" % (self.__class__.__name__, name))
AttributeError: 'DataFrame' object has no attribute 'shape'



In [89]:
df_final=tfidf_df.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [90]:
df_final['positive_negative']=positive_negative

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [91]:
df_final.select('tfidf')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'DataFrame' object has no attribute 'select'
Traceback (most recent call last):
  File "/tmp/1648906651298-0/lib/python3.7/site-packages/pandas/core/generic.py", line 5179, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'DataFrame' object has no attribute 'select'



In [92]:
dataframe = spark.createDataFrame(df_final)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [93]:
dataframe.select('positive_negative')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[positive_negative: bigint]

In [94]:
dataframe.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- text: string (nullable = true)
 |-- label: long (nullable = true)
 |-- words: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- words_nsw: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- tf: vector (nullable = true)
 |-- tfidf: vector (nullable = true)
 |-- positive_negative: long (nullable = true)

In [95]:
splits = dataframe.select(['tfidf', 'positive_negative']).randomSplit([0.8,0.2],seed=100)
train = splits[0].cache()
test = splits[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [108]:
train.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------------+
|               tfidf|positive_negative|
+--------------------+-----------------+
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                1|
|(19144,[0,1,2,3,4...|                1|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                1|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                1|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                1|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                1|
|(19144,[0,1,2,3,4...|                0|
|(19144,[0,1,2,3,4...|                0|
+--------------------+-----------------+
only showing top

In [96]:
from pyspark.mllib.classification import SVMModel, SVMWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors as MLLibVector

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [97]:
# convert to LabeledPoint vectors
train_lb = train.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))
test_lb = test.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [98]:
# SVM model
numIterations = 50
regParam = 0.3
svm = SVMWithSGD.train(train_lb, numIterations, regParam=regParam)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [99]:
# predict
test_lb = test.rdd.map(lambda row: LabeledPoint(row[1], MLLibVectors.fromML(row[0])))
scoreAndLabels_test = test_lb.map(lambda x: (float(svm.predict(x.features)), x.label))
score_label_test = spark.createDataFrame(scoreAndLabels_test, ["prediction", "positive_negative"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [101]:
score_label_test.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----------------+
|prediction|positive_negative|
+----------+-----------------+
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              1.0|
|       0.0|              1.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              1.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
|       0.0|              0.0|
+----------+-----------------+
only showing top 20 rows

In [103]:
f1_eval = MulticlassClassificationEvaluator(labelCol="positive_negative", predictionCol="prediction", metricName="f1")
svm_f1 = f1_eval.evaluate(score_label_test)
print("F1 score: %.4f" % svm_f1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 score: 0.7876

In [111]:
new_review = 'It s like the Walmart of sports merchandise  The prices aren t that great  it s out in the boonies  it s crowded  and Nike gear isn t that great  Some people obviously like it  but it was not for me '
new_review = re.sub('[^a-zA-Z]', ' ', new_review)
new_review = new_review.lower()
new_review = new_review.split()
ps = PorterStemmer()
all_stopwords = stopwords.words('english')
all_stopwords.remove('not')
new_review = [ps.stem(word) for word in new_review if not word in set(all_stopwords)]
new_review = ' '.join(new_review)
new_corpus = [new_review]
new_X_test = cv.transform(new_corpus).toarray()
new_y_pred = classifier.predict(new_X_test)
print(new_y_pred)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'CountVectorizer' object has no attribute 'transform'
Traceback (most recent call last):
AttributeError: 'CountVectorizer' object has no attribute 'transform'



In [ ]:
# count vectorizer
def predict(text):
    cv = CountVectorizer(inputCol='words_nsw', outputCol='tf')
    cvModel = cv.fit(review_tokenized)
    count_vectorized = cvModel.transform(review_tokenized)
    
    

In [ ]:
#########################################################################################################

In [110]:
#########logistic Regression#####################

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…